In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

In [3]:
torch.__version__


'2.3.1'

In [349]:
#hypyer parameters 
batch_size = 64
block_size = 256
max_iters = 5000

eval_interval = 500
learning_rate = 3e-4
device ='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

n_embd = 384
n_head = 6 
n_layer = 6
dropout = 0.2

In [8]:
torch.manual_seed(1337)

In [9]:
with open('../input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [22]:
# crate vocabulary list (sorted)
char = sorted(list(set(text)))
vocab_size = len(char)
vocab_size

65

In [28]:
stoi = {char: i for i, char in enumerate(char)}
itos = {i: char for i, char in enumerate(char)}


In [38]:
b = encoding('fuck you!')
b

[44, 59, 41, 49, 1, 63, 53, 59, 2]

In [65]:
encoding = lambda word: [stoi[i] for i in word]
# use '' to concatenate str 
decoding = lambda number: ''.join(itos[i] for i in number)

In [97]:
#training ans tesing split 
data = torch.tensor(encoding(text), dtype=torch.int64)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [201]:
# Data loading 
# Batch in training and validation
def get_batch(set_name):
    data = (train_data if set_name == 'train' else  val_data)
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    
    return x, y

In [267]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out 
        

In [350]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B, T, C = x.shape
#         X W + b 
        k = self.key(x)
        q = self.query(x)
        # attention score 
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5 # aiming to control it between 0 - 1 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        # weighted sum ---? weights @ value (x)
        out = wei @ v # (B, T, hs)
        return out

In [392]:
#multihead attention
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init_()
        # define a group of heads (nn.ModuleList including multiple head instance)
        self.heads = nn.ModuleList([Head(head_size)for _ in range(num_heads)])
        
        self.proj = nn.Linear(head_size * num_heads, n_embd) 
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torcg.cat([h(x) for h in self.heads], dim=-1)
        # here's how we make num_heads * head_size to -->>>>> n_embd 
        out = self.dropout(selfl.proj(out))

In [ ]:
# embedding x --> B, T, C
